In [1]:
import numpy as np
np.random.seed(1337)

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Lambda, Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from sklearn import metrics
from keras.regularizers import WeightRegularizer
import time

Using Theano backend.


In [26]:

dummy_empty = 2 ** 17

context_train = []
ad_train = []
y_train = []


with open('../../data/click-all-train.thumb') as f:
    for line in f:
        label, context, a,b,c,d, ad, e,f,g,h = line.split("\t")
        y_train.append(int(label))
        context_train.append(map(lambda x: int(x) if x else dummy_empty, context.strip().split(" ")))
        ad_train.append(map(lambda x: int(x) if x else dummy_empty, ad.strip().split(" ")))



context_test = []
ad_test = []
y_test = []

with open('../../data/click-all-test.thumb') as f:
    label, context_name, a,b,c,d, ad_name, c,f,g,h = line.split("\t")
    y_test.append(int(label))
    context_test.append(map(lambda x: int(x) if x else dummy_empty, context_name.strip().split(" ")))
    ad_test.append(map(lambda x:  int(x) if x else dummy_empty, ad_name.strip().split(" ")))
    
with open('../../data/click-all-test.thumb') as f:
    for line in f:
        label, context, a,b,c,d,ad ,e,f,g,h= line.strip().split("\t")
        y_test.append(int(label))
        context_test.append(map(lambda x: int(x) if x else dummy_empty, context.strip().split(" ")))
        ad_test.append(map(lambda x: int(x) if x else dummy_empty, ad.strip().split(" ")))


In [7]:

dummy_empty = 2 ** 17

context_train = []
ad_train = []
y_train = []
with open('../../data/click-all-train.thumb') as f:
    for line in f:
        label, context,a,b,c,d, ad,e,f,g,h = line.strip().split("\t")
        y_train.append(int(label))
        context_train.append(map(lambda x: int(x) if x else dummy_empty, context.strip().split(" ")))
        ad_train.append(map(lambda x: int(x) if x else dummy_empty, ad.strip().split(" ")))
        
        
context_test = []
ad_test = []
y_test = []
with open('../../data/click-all-test.thumb') as f:
    for line in f:
        label, context, a,b,c,d,ad ,e,f,g,h= line.strip().split("\t")
        y_test.append(int(label))
        context_test.append(map(lambda x: int(x) if x else dummy_empty, context.strip().split(" ")))
        ad_test.append(map(lambda x: int(x) if x else dummy_empty, ad.strip().split(" ")))

In [27]:

maxlen = 15


context_train = sequence.pad_sequences(context_train, maxlen=maxlen)

ad_train = sequence.pad_sequences(ad_train, maxlen=maxlen)


context_test = sequence.pad_sequences(context_test, maxlen=maxlen)


ad_test = sequence.pad_sequences(ad_test, maxlen=maxlen)


In [28]:
def means(x):
    return K.mean(x, axis=1)

In [29]:
from keras.layers import Merge

max_features = 2 ** 17 + 1
batch_size = 32
embedding_dims = 3


context_name = Sequential()
context_name.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
context_name.add(Dropout(0.5))
context_name.add(Lambda(means, output_shape=(embedding_dims,)))


ad_name = Sequential()
ad_name.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
ad_name.add(Dropout(0.5))
ad_name.add(Lambda(means, output_shape=(embedding_dims,)))

merged = Merge([context_name, ad_name], mode='concat')

final_model = Sequential()
final_model.add(merged)

final_model.add(Dense(20))
final_model.add(BatchNormalization())
final_model.add(LeakyReLU(alpha=.01))
final_model.add(Dropout(0.5))
final_model.add(Dense(1))
final_model.add(BatchNormalization())
final_model.add(Activation('sigmoid'))

final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['fmeasure'])

model_path = ''
for i in range(10):
    if model_path != '':
        final_model.load_weights(model_path)
    model_path = '../../model/weights.epoch' + str(i) + "_" + str(int(time.time())) + '.hdf5'
    weight_save_callback = ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto')
    final_model.fit([context_train, ad_train], y_train, batch_size=batch_size, validation_data=([context_test,ad_test], y_test), nb_epoch=1, callbacks=[weight_save_callback])
    
    y_pred = final_model.predict([context_train, ad_train])[:, 0]
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred)
    print "train auc -> " + str(metrics.auc(fpr, tpr))
    
    y_pred = final_model.predict([context_test, ad_test])[:, 0]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    print "test auc -> " + str(metrics.auc(fpr, tpr))



Train on 10000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 1s - loss: 0.7760 - fmeasure: 0.4777 - val_loss: 0.7049 - val_fmeasure: 0.0000e+00
train auc -> 0.63756744687
test auc -> 0.513149960587
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 1s - loss: 0.6896 - fmeasure: 0.5723 - val_loss: 0.6930 - val_fmeasure: 0.0080
train auc -> 0.794927224742
test auc -> 0.524383080118
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 1s - loss: 0.6304 - fmeasure: 0.6499 - val_loss: 0.7016 - val_fmeasure: 0.6203
train auc -> 0.877940566689
test auc -> 0.53582632773
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 2s - loss: 0.5907 - fmeasure: 0.7010 - val_loss: 0.7179 - val_fmeasure: 0.5813
train auc -> 0.928460413397
test auc -> 0.530473173308
Train on 10000 samples, validate o

ValueError: I/O operation on closed file

In [6]:
proc = subprocess.Popen('./fasttext print-vectors ~/Downloads/typeid_areaid_poiid_corpus_20161206_night.bin < test_seg.txt', shell=True, stdout=subprocess.PIPE)
arr = np.loadtxt(proc.stdout)

txt = []
with open('test_seg.txt', 'r') as f:
    for line in f:
        txt.append(line.rstrip().decode('utf-8'))

In [ ]:
import matplotlib.pyplot as plt
import mpld3
 


from matplotlib.font_manager import FontProperties

mpld3.enable_notebook()

ChineseFont1 = FontProperties(fname = '/System/Library/Fonts/STHeiti Medium.ttc')
 

# wv = arr
# vocabulary = txt

plt.scatter(Y[:, 0], Y[:, 1])
for label, x, y in zip(vocabulary, Y[:, 0], Y[:, 1]):
    # plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontproperties=ChineseFont1)
    plt.text(x, y, label)
    


Y = Y[:60, :]


plt.scatter(Y[:, 0], Y[:, 1])
for label, x, y in zip(vocabulary, Y[:, 0], Y[:, 1]):
    # plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontproperties=ChineseFont1)
    plt.text(x, y, label)
# mpld3.display()



mpld3.show()


Note: if you're in the IPython notebook, mpld3.show() is not the best command
      to use. Consider using mpld3.display(), or mpld3.enable_notebook().
      See more information at http://mpld3.github.io/quickstart.html.

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [07/Dec/2016 16:20:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2016 16:20:14] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2016 16:20:14] "GET /mpld3.js HTTP/1.1" 200 -



Note: if you're in the IPython notebook, mpld3.show() is not the best command
      to use. Consider using mpld3.display(), or mpld3.enable_notebook().
      See more information at http://mpld3.github.io/quickstart.html.

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [07/Dec/2016 16:18:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2016 16:18:36] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2016 16:18:36] "GET /mpld3.js HTTP/1.1" 200 -



stopping Server...
